In [ ]:
--Dataset: Northwind database
--Source: https://github.com/jpwhite3/northwind-SQLite3?tab=readme-ov-file


-- Popular products per category

SELECT
    productName,
    MAX(UnitsOnOrder) AS no_units_ordered,
    MAX(ReorderLevel) AS Reodered_frequency
FROM Products p
JOIN Categories c
    ON c.CategoryID=p.CategoryID
JOIN 'Order Details' o
    ON p.ProductID=o.ProductID
GROUP BY p.productID
HAVING MAX(UnitsOnOrder) > 0 AND MAX(ReorderLevel) > 0
______________________________________________________

-- How many orders were placed in December 2016?
SELECT
    COUNT(orderID) AS Order_Count
FROM Orders
WHERE OrderDate BETWEEN '2016-12-01' AND '2016-12-31'

______________________________________________________

-- Report for item with order_id = 1024
-- Showing product name, unit price, quantity, and supplier name

SELECT
    p.productName,
    o.unitPrice,
    o.quantity,
    s.companyName as supplier_name
FROM products as p
JOIN 'order Details' as o
	ON p.productID=o.productID
JOIN suppliers as s
	ON s.supplierID=p.supplierID
WHERE orderID = 10248
______________________________________________________

-- Report for each product
SELECT
    p.productName,
    s.companyName,
    c.categoryName,
    p.unitPrice,
    p.quantityPerUnit
FROM products as p
JOIN suppliers as s
	ON s.supplierID=p.supplierID
JOIN categories as c
	ON c.categoryID=p.categoryID
______________________________________________________

 -- Report of price of item with order_id = 10250 Before and After discount
SELECT
  SUM(oi.unitPrice * oi.quantity) AS total_price,
  SUM(unitPrice * quantity * (1 - discount)) AS total_price_after_discount
FROM orders o
JOIN 'order Details' oi
  ON o.orderID = oi.orderID
WHERE o.orderID = 10250;

______________________________________________________

-- Revenue including and excluding the discount for Year 2016 per Shipped Country
SELECT
    ShipCountry,
    COUNT(o.OrderID) AS Total_ordres,
    ROUND(SUM(UnitPrice * quantity *(1-discount)),2) AS Revenue_after_discount,
    ROUND(SUM(UnitPrice * quantity),2) AS Revenue_before_discount
FROM Orders o
JOIN 'Order Details' od
    ON o.OrderID=od.OrderID
WHERE ShippedDate BETWEEN '2016-01-01' AND '2017-01-01'
GROUP BY ShipCountry

______________________________________________________

-- Report of number of employees hired in 2013

SELECT COUNT(*) as number_of_employees
FROM employees
WHERE HireDate > '2013-01-01' AND hireDate < '2013-12-31'


______________________________________________________
-- How many orders were processed by each employee?

SELECT
    e.employeeID,
    e.firstName,
    e.lastName,
    COUNT(o.employeeID) as orders_count
FROM employees as e
JOIN orders as o
	ON e.employeeID=o.employeeID
GROUP BY e.employeeID, e.firstName

______________________________________________________
-- How much do customers (companies) order from us?

SELECT
    c.customerID,
    c.companyName,
    COUNT(*) AS orders_count
FROM customers as c
JOIN orders as o
	ON c.customerID=o.customerID
GROUP BY c.customerID, c.companyName

______________________________________________________
-- Showing which customers paid the most for orders made in June 2016 or July 2016.

SELECT
  c.companyName,
  ROUND(SUM(unitPrice * quantity * (1 - discount)),2) AS total_paid
FROM orders o
JOIN 'order Details' oi
  ON o.orderID = oi.orderID
JOIN customers c
  ON o.customerID = c.customerID
WHERE orderDate >= '2016-06-01' AND orderDate < '2016-08-01'
GROUP BY c.customerID,
  c.companyName
ORDER BY total_paid DESC
______________________________________________________

/* The goal of this SQL code is to show the percentage of total revenue (before discount)
generated by orders shipped to the USA and to Germany for each employee with respect to the total
revenue generated by that employee.*/

-- 1st CTE to calculate revenue for USA
WITH usa_revenue AS (
  SELECT
    EmployeeID,
    SUM(CASE WHEN ShipCountry = 'USA' THEN UnitPrice * quantity ELSE 0 END) AS country_revenue,
    SUM(UnitPrice * quantity) AS total_revenue
  FROM orders o
  JOIN 'Order Details' oi
    ON o.OrderID = oi.OrderID
  GROUP BY EmployeeID),

-- 2nd CTE to calculate revenue for GERMANY
germany_revenue AS (
  SELECT
    EmployeeID,
    SUM(CASE WHEN ShipCountry = 'Germany' THEN UnitPrice * quantity ELSE 0 END) AS country_revenue,
    SUM(UnitPrice * quantity) AS total_revenue
  FROM orders o
  JOIN 'Order Details'  oi
    ON o.OrderID = oi.OrderID
  GROUP BY EmployeeID)

-- Outer query to calculate the percentage per employee
SELECT
  e.EmployeeID,
  e.FirstName,
  e.LastName,
  ROUND(us_rev.country_revenue / us_rev.total_revenue * 100, 2) AS rev_percentage_usa,
  ROUND(ger_rev.country_revenue / ger_rev.total_revenue * 100, 2) AS rev_percentage_germany
FROM usa_revenue us_rev
JOIN germany_revenue ger_rev
  ON us_rev.EmployeeID = ger_rev.EmployeeID
JOIN employees e
  ON e.EmployeeID = us_rev.EmployeeID
